## hey 

In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_columns', 5)
pd.set_option('display.max_rows', 5)

In [5]:
df = pd.read_csv('../data/benin-malanville.csv')

In [6]:
df.head()

,Timestamp,GHI,...,TModB,Comments
0,2021-08-09 00:01,-1.2,...,26.2,NaN
1,2021-08-09 00:02,-1.1,...,26.2,NaN
2,2021-08-09 00:03,-1.1,...,26.2,NaN
3,2021-08-09 00:04,-1.1,...,26.3,NaN
4,2021-08-09 00:05,-1.0,...,26.3,NaN
